# Digit Recognition App

To run: Kernel -> Restart & Run All

In [1]:
import ipywidgets as w
import pandas as pd

from SimpleCV import *
from ipywidgets import widgets, HBox, VBox
from IPython.display import display
from sklearn.externals import joblib

disp = Display(displaytype='notebook')
app = w.Output()

FILE_NAME = None
IMAGES = None
FILES = []

### Importing Normalizer & Classifier

In [2]:
CLASSIFIER = joblib.load('classifier_svm_v6.joblib')
SCALER = joblib.load('scaler_svm_v6.joblib')
COLUMNS = [u'area', u'contours', u'radius', u'hull_radius', u'centroid_x',
       u'centroid_y', u'angle', u'weight_0_2', u'weight_0_3', u'weight_0_4',
       u'weight_0_5', u'weight_0_6', u'weight_1_1', u'weight_1_2',
       u'weight_1_3', u'weight_1_4', u'weight_1_5', u'weight_1_6',
       u'weight_2_1', u'weight_2_2', u'weight_2_3', u'weight_2_4',
       u'weight_2_5', u'weight_2_6', u'weight_3_1', u'weight_3_2',
       u'weight_3_3', u'weight_3_4', u'weight_3_5', u'weight_3_6',
       u'weight_4_1', u'weight_4_2', u'weight_4_3', u'weight_4_4',
       u'weight_4_5', u'weight_4_6', u'weight_5_1', u'weight_5_2',
       u'weight_5_3', u'weight_5_4', u'weight_5_5', u'weight_5_6',
       u'weight_6_1', u'weight_6_2', u'weight_6_3', u'weight_6_4',
       u'weight_6_5', u'weight_6_6', u'weight_7_2', u'weight_7_3',
       u'weight_7_4', u'weight_7_5', u'weight_7_6', u'weight_7_7', u'num_holes']
COLUMNS_TO_NORMALIZE = [u'area', u'contours', u'radius', u'hull_radius', u'centroid_x',
       u'centroid_y', u'angle', u'weight_0_2', u'weight_0_3', u'weight_0_4',
       u'weight_0_5', u'weight_0_6', u'weight_1_1', u'weight_1_2',
       u'weight_1_3', u'weight_1_4', u'weight_1_5', u'weight_1_6',
       u'weight_2_1', u'weight_2_2', u'weight_2_3', u'weight_2_4',
       u'weight_2_5', u'weight_2_6', u'weight_3_1', u'weight_3_2',
       u'weight_3_3', u'weight_3_4', u'weight_3_5', u'weight_3_6',
       u'weight_4_1', u'weight_4_2', u'weight_4_3', u'weight_4_4',
       u'weight_4_5', u'weight_4_6', u'weight_5_1', u'weight_5_2',
       u'weight_5_3', u'weight_5_4', u'weight_5_5', u'weight_5_6',
       u'weight_6_1', u'weight_6_2', u'weight_6_3', u'weight_6_4',
       u'weight_6_5', u'weight_6_6', u'weight_7_2', u'weight_7_3',
       u'weight_7_4', u'weight_7_5', u'weight_7_6', u'weight_7_7']

### Image Selection Logic

In [3]:
def on_path_select(b):
    global FILES
    FILES = sorted(glob.glob(os.path.join(input_path.value, "*.png")))
    dropdown_image.options = FILES

def on_image_select(b):
    global FILE_NAME
    FILE_NAME = dropdown_image.value
    if FILE_NAME:
        # preview UI
        image_preview.value = w.Image.from_file(FILE_NAME).value
        # reset prediction UI
        reset_labels()

        get_individual_images(FILE_NAME)
        
        # update preview
        if IMAGES:
            for i in range(4):
                digit_previews[i].clear_output()
                with digit_previews[i]:
                    IMAGES[i][1].resize(74, 74).save(disp)

### Image Separation Logic

In [4]:
# Constant single digit image size
SIZE = 32

# Return an array of individual images and their digit from the image object
def get_individual_images(file_name):
    global IMAGES
    img = Image(str(file_name))
#     if img.size == (128, 32):
    IMAGES = [
        get_image_obj(crop(img, 0)), 
        get_image_obj(crop(img, 1)), 
        get_image_obj(crop(img, 2)), 
        get_image_obj(crop(img, 3))
    ]

def get_image_obj(img):
    _bin = img.binarize()
    return (None, _bin.dilate().erode(), _bin)

# Crop and return a part of the source image
def crop(img, part):
    return img.crop(SIZE * part, 0, SIZE, SIZE)

### Feature Extraction Logic

In [5]:
def get_weighted_matrix(img):
    COUNT = 8 # GRID SIZE
    T = 5 # THRESHOLD
    CROP = SIZE / COUNT
    m = np.zeros((COUNT, COUNT))
    for y in range(COUNT):
        for x in range(COUNT):
            part = img.crop(x * CROP, y * CROP, CROP, CROP)
            sum = (part.getNumpy()[:,:,0] / 255).sum()
            m[x][y] = sum if sum > T else 0
    return m.T

def extract_features(image):
    # Columns detected using computer vision
    features_dict = {
        "area": [],
        "contours": [],
        "radius": [],
        "circle_dist": [],
        "rect_dist": [],
        "hull_radius": [],
        "aspect_ratio": [],
        "centroid_x": [],
        "centroid_y": [],
        "angle": [],
        "num_holes": []
    }

    column_order = [
        "area",
        "contours",
        "radius",
        "circle_dist",
        "rect_dist",
        "hull_radius",
        "aspect_ratio",
        "centroid_x",
        "centroid_y",
        "angle"
    ]

    # Adding column names for the low-res pixel count matrix
    COUNT = 8
    for x in range(COUNT):
        for y in range(COUNT):
            name = '_'.join(['weight', str(x), str(y)])
            column_order.append(name)
            features_dict[name] = []

    column_order.append("num_holes")
    
    # HACK TO FIND BLOBS
    image[1].show()
    # Find blobs in the preprocessed image
    blobs = image[1].findBlobs()
    if blobs:
        blob = blobs[0]
        features_dict['area'].append(blob.area())
        features_dict['contours'].append(len(blob.contour()))
        features_dict['circle_dist'].append(blob.circleDistance())
        features_dict['rect_dist'].append(blob.rectangleDistance())
        features_dict['radius'].append(blob.radius())
        features_dict['hull_radius'].append(blob.hullRadius())
        features_dict['aspect_ratio'].append(blob.aspectRatio())
        features_dict['centroid_x'].append(blob.centroid()[0])
        features_dict['centroid_y'].append(blob.centroid()[1])
        features_dict['angle'].append(blob.angle())

        # Calculate matrix in the preprocessed image
        m = get_weighted_matrix(image[1])
        COUNT = 8
        for x in range(COUNT):
            for y in range(COUNT):
                features_dict['_'.join(['weight', str(x), str(y)])].append(m[x, y])

        # Find the holes in the 'just binarized' image
        num_holes = len(image[2].embiggen(2).invert().findBlobs()[:-1])
        features_dict['num_holes'].append(num_holes)
    
    return pd.DataFrame(features_dict, columns=column_order)

### Prediction Logic

In [6]:
def predict(index):
    if IMAGES:
        # extract features
        features = extract_features(IMAGES[index])[COLUMNS]
        if len(features) == 1:
            # normalize
            features[COLUMNS_TO_NORMALIZE] = pd.DataFrame(SCALER.transform(features[COLUMNS_TO_NORMALIZE]), 
                                                          columns=COLUMNS_TO_NORMALIZE)
            # predict
            prediction = CLASSIFIER.predict(features)[0]
            # update UI
            set_label(index, prediction)
            return prediction
    set_label(index, '-')
    return None

def predict_all(b):
    return [predict(0), predict(1), predict(2), predict(3)]

### Image Selection UI

In [7]:
input_path = w.Text(value="../dataset-images/", placeholder="path", description="Image folder:")
button_select_path = w.Button(description='Use')
button_select_path.on_click(on_path_select)

dropdown_image = w.Dropdown(description='Image:')
button_select_image = w.Button(description="Use")
button_select_image.on_click(on_image_select)
image_preview = w.Image(layout={'width': '340px'})

image_selection = VBox([
    HBox([input_path, button_select_path]),
    HBox([dropdown_image, button_select_image])
])

with app:
    display(image_selection)
    display(image_preview)

### Prediction UI

In [8]:
digit_previews = [w.Output(), w.Output(), w.Output(), w.Output()]

layout_predict_button = {'width': '82px'}
buttons_predict = [
    w.Button(description="Predict", layout=layout_predict_button),
    w.Button(description="Predict", layout=layout_predict_button),
    w.Button(description="Predict", layout=layout_predict_button),
    w.Button(description="Predict", layout=layout_predict_button)
]

layout_predict_label = {'margin': '0 auto'}
labels_predict = [
    w.Label(layout=layout_predict_label),
    w.Label(layout=layout_predict_label),
    w.Label(layout=layout_predict_label),
    w.Label(layout=layout_predict_label)
]

def reset_labels(value='-'):
    set_label(0, value)
    set_label(1, value)
    set_label(2, value)
    set_label(3, value)

def set_label(index, value):
    labels_predict[index].value = str(value)
    
buttons_predict[0].on_click(lambda b: predict(0))
buttons_predict[1].on_click(lambda b: predict(1))
buttons_predict[2].on_click(lambda b: predict(2))
buttons_predict[3].on_click(lambda b: predict(3))

button_predict_all = w.Button(description="Predict Postal Code", layout={'width': '340px'})
button_predict_all.on_click(predict_all)

selection = VBox([
    HBox([
        VBox([digit_previews[0], labels_predict[0], buttons_predict[0]]),
        VBox([digit_previews[1], labels_predict[1], buttons_predict[1]]),
        VBox([digit_previews[2], labels_predict[2], buttons_predict[2]]),
        VBox([digit_previews[3], labels_predict[3], buttons_predict[3]])
    ]),
    button_predict_all
])
    
with app:
    display(selection)
    
    # Initial situation
    on_path_select(None)
    on_image_select(None)

# Digit Recognition App

### How to use:

#### Predict a single postal code

1. Select a folder path.
2. Select a 128x32 pixel 'postal code' image to recognize.
3. Predict individual digits or the whole postal code at once.

#### Predict all postal codes

1. Select a folder path.
2. Predict all images in the folder.

In [9]:
app

Output()